In [ ]:
# Import libraries
import io
from pathlib import Path
import zipfile
import pandas as pd

# Define directories
DATA_DIR = Path("data")

# Clean traffic volume data
for zip_path in sorted(DATA_DIR.glob("*.zip")):

    year = str(zip_path)[-8:-4]
    dfs = []

    with zipfile.ZipFile(zip_path) as zip_1:
        for zip_name in sorted(zip_1.namelist()):
            with zip_1.open(zip_name) as zip_file:
                with zipfile.ZipFile(io.BytesIO(zip_file.read())) as zip_2:
                    for i, csv_name in enumerate(sorted(zip_2.namelist())):
                        if csv_name.lower().endswith(".csv"):
                            with zip_2.open(csv_name) as csv_file:

                                df = pd.read_csv(csv_file)

                                # Drop irrelavant columns
                                df = df.drop(["NM_REGION", "QT_VOLUME_24HOUR", "CT_ALARM_24HOUR"], axis=1)

                                # Rename column names
                                df = df.rename(columns={"NB_SCATS_SITE": "site_id", 
                                                        "QT_INTERVAL_COUNT": "datetime",
                                                        "NB_DETECTOR": "detector_id", 
                                                        "CT_RECORDS": "working_period_count"})
                                
                                # Remove rows with all non-positive volume values
                                volume_cols = df.columns[3:99]
                                df = df[(df[volume_cols] > 0).any(axis=1)]

                                # Convert dtypes
                                df["site_id"] = df["site_id"].astype("int16")
                                df["datetime"] = pd.to_datetime(df["datetime"])
                                df['detector_id'] = df["detector_id"].astype("int8")
                                df['working_period_count'] = df["working_period_count"].astype("int8")
                                # df[volume_cols] = df[volume_cols].astype("int16")

                                dfs.append(df.iloc[1:] if i > 0 else df)
                                print(f"Cleaned and added {csv_name} successfully.")
            break
                        
    traffic_df = pd.concat(dfs, ignore_index=True)
    break

Cleaned and added VSDATA_202001/VSDATA_20200101.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200102.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200103.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200104.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200105.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200106.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200107.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200108.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200109.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200110.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200111.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200112.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200113.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200114.csv successfully.
Cleaned and added VSDATA_202001/VSDATA_20200115.csv successfully.
Cleaned an

In [10]:
# traffic_df.groupby("site_id").count()

traffic_df[traffic_df["site_id"] == 100][traffic_df["detector_id"] == 1]

/tmp/ipykernel_691/3530363808.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  traffic_df[traffic_df["site_id"] == 100][traffic_df["detector_id"] == 1]


,site_id,datetime,detector_id,V00,V01,V02,V03,V04,V05,V06,...,V87,V88,V89,V90,V91,V92,V93,V94,V95,working_period_count
88394,100,2020-01-01,1,5,5,3,6,13,7,5,...,7,7,6,3,3,3,2,2,2,96
183377,100,2020-01-02,1,2,1,1,2,2,1,1,...,12,7,3,11,5,3,3,0,3,96
278357,100,2020-01-03,1,2,2,1,4,0,1,1,...,9,14,5,9,10,7,7,3,6,96
373596,100,2020-01-04,1,4,5,2,2,1,1,4,...,5,4,11,6,6,7,1,9,1,96
469255,100,2020-01-05,1,4,3,4,1,1,1,4,...,14,1,7,4,4,6,2,4,1,96
564295,100,2020-01-06,1,4,2,0,0,1,1,1,...,11,4,8,2,5,2,3,2,3,96
655948,100,2020-01-07,1,1,3,2,1,3,0,2,...,9,16,15,11,3,1,2,3,5,96
750846,100,2020-01-08,1,9,1,2,0,1,0,1,...,8,11,12,13,4,6,6,5,6,96
845802,100,2020-01-09,1,1,3,0,4,1,0,1,...,12,5,9,3,5,2,5,3,3,96
939797,100,2020-01-10,1,3,3,2,2,2,0,1,...,5,4,8,9,5,6,3,3,4,96


In [ ]:
# Including rows with zeros: ~3 million 
# Not including: 400k